# Random forest importance

In [1]:
%store -r X_256
%store -r y_256
%store -r X_test_256
%store -r y_test_256
%store -r feature_names_256

In [2]:
# Import thư viện
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [3]:
import matplotlib.pyplot as plt

# Reshape data
# Data ban đầu X(13166, 20, 128), y(13166)

# Reshape X về dạng 2d => X_2d(263320, 128), giữ nguyên 128 và đó là các đặc trưng
X_2d = X_256.reshape(-1, X_256.shape[-1])
# Reshape y thành y_reshaped(263320) bằng cách lặp mỗi phần tử trong y 20 lần
y_reshaped = np.repeat(y_256, 20)

In [4]:
# Khởi tạo mô hình Random Forest
model = RandomForestClassifier(n_estimators=500)
# Nó chỉ định số lượng cây quyết định sẽ được xây dựng và kết hợp để tạo thành khu rừng ngẫu nhiên trong thuật toán random forest.

In [5]:
# Huấn luyện mô hình
model.fit(X_2d, y_reshaped)

RandomForestClassifier(n_estimators=500)

In [6]:
# Lấy tầm quan trọng của đặc trưng
feature_importances = model.feature_importances_

# In tầm quan trọng của đặc trưng
print(feature_importances)

[0.01464454 0.01180205 0.01175091 0.01104091 0.01150786 0.01175087
 0.01131469 0.01138003 0.01123316 0.01122646 0.01159382 0.01109325
 0.01576134 0.02852493 0.02869856 0.02975458 0.02208632 0.03019507
 0.02126903 0.02183888 0.01398782 0.01716393 0.01445777 0.01648567
 0.01397774 0.01269454 0.01285129 0.01406269 0.01045486 0.00697223
 0.00663724 0.00371894 0.00386907 0.00347711 0.00289173 0.0031168
 0.00284471 0.00264199 0.00295568 0.00334703 0.00352515 0.00286662
 0.00337169 0.00312729 0.0026896  0.0031121  0.00256771 0.00284167
 0.00283459 0.00272066 0.00241028 0.00269078 0.00314855 0.00275559
 0.00358605 0.00349857 0.00311634 0.00383338 0.00397683 0.00436359
 0.00526375 0.0051167  0.00547799 0.00410326 0.00341626 0.0039258
 0.00358378 0.00309512 0.00352633 0.00266487 0.00310671 0.00370116
 0.0034988  0.00334545 0.00378893 0.00255436 0.00265465 0.00306352
 0.00365643 0.00381907 0.00363408 0.00379802 0.00335903 0.00337821
 0.00315826 0.00303594 0.00216679 0.00226288 0.00196649 0.002224

In [7]:
feature_importances.shape

(256,)

In [8]:
feature_names_256

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '246', '247', '248', '249', '250', '251', '252', '253', '254', '255'],
      dtype='object', length=256)

In [9]:
# Tạo dataframe gồm tên đặc trưng và tầm quan trọng
feature_importances_df = pd.DataFrame({"feature_name": feature_names_256, "importance": feature_importances})

# Sắp xếp dataframe theo tầm quan trọng
feature_importances_df = feature_importances_df.sort_values("importance", ascending=False)

# Lấy index của dataframe
indices = feature_importances_df.index

# In index
print(indices)

Index([ 17,  15,  14,  13,  16,  19,  18,  21,  23,  12,
       ...
       208, 217, 190, 201, 207, 200, 220, 199, 210, 212],
      dtype='int64', length=256)


In [10]:
# Xác định số đặc trưng sẽ giữ lại để train  model
k = 128
top_k_indices = indices[:k]

In [11]:
# Giữ lại k đặc trưng tốt nhất trong tệp dữ liệu ban đầu
selected_X = X_256[:,:,top_k_indices]
selected_X_test = X_test_256[:,:,top_k_indices]

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Khởi tạo mô hình
model = keras.Sequential()

model.add(layers.Input(shape = (20, k, 1)))

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 5x5 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (5, 5), activation='relu', padding='same'))

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 5x5 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (5,5), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Flatten để làm phẳng đầu ra của lớp trước khi đi vào các lớp fully connected
model.add(layers.Flatten())

# Thêm lớp fully connected (Dense Layer) với 64 units và hàm kích hoạt ReLU
model.add(layers.Dense(64, activation='relu'))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp fully connected (Dense Layer) cuối cùng với số lớp đầu ra phụ thuộc vào bài toán của bạn
model.add(layers.Dense(5, activation='softmax'))

# In thông tin mô hình
model.summary()

# Biên dịch mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 20, 256, 32)         │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 20, 256, 32)         │          25,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 10, 128, 32)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 10, 128, 32)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 10, 128, 64)         │          51,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 10, 128, 64)         │         102,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 5, 64, 64)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 20480)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       1,310,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             325 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,491,301 (5.69 MB)

 Trainable params: 1,491,301 (5.69 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.fit(selected_X, y_256, epochs=10, batch_size=16)

Epoch 1/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 132s 156ms/step - accuracy: 0.4446 - loss: 1.2906
Epoch 2/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 129s 156ms/step - accuracy: 0.6598 - loss: 0.8099
Epoch 3/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 131s 160ms/step - accuracy: 0.7153 - loss: 0.6787
Epoch 4/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 125s 152ms/step - accuracy: 0.7604 - loss: 0.5859
Epoch 5/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 134s 162ms/step - accuracy: 0.7682 - loss: 0.5720
Epoch 6/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 125s 152ms/step - accuracy: 0.7981 - loss: 0.5215
Epoch 7/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 134s 162ms/step - accuracy: 0.8007 - loss: 0.5083
Epoch 8/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 125s 152ms/step - accuracy: 0.8082 - loss: 0.4888
Epoch 9/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 132s 160ms/step - accuracy: 0.8210 - loss: 0.4674
Epoch 10/10
823/823 ━━━━━━━━━━━━━━━━━━━━ 126s 154ms/step - accuracy: 0.8237 - loss: 0.4629


In [15]:
# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = model.evaluate(selected_X_test, y_test_256)
print("Độ chính xác trên tập kiểm tra:", test_accuracy)

# Dự đoán các nhãn trên tập kiểm tra
y_pred = model.predict(selected_X_test)

# Chuyển đổi dự đoán thành nhãn dự đoán (lớp dự đoán)
y_pred_classes = np.argmax(y_pred, axis=1)

# Hiển thị ma trận nhầm lẫn
from sklearn.metrics import confusion_matrix, classification_report
confusion = confusion_matrix(y_test_256, y_pred_classes)
print("Ma trận nhầm lẫn:")
print(confusion)

# Hiển thị báo cáo phân loại
class_report = classification_report(y_test_256, y_pred_classes)
print("Báo cáo phân loại:")
print(class_report)

412/412 ━━━━━━━━━━━━━━━━━━━━ 34s 82ms/step - accuracy: 0.8998 - loss: 0.2158
Độ chính xác trên tập kiểm tra: 0.908932089805603
412/412 ━━━━━━━━━━━━━━━━━━━━ 34s 82ms/step
Ma trận nhầm lẫn:
[[2842    0    0    0    0]
 [   1 1974   21    0   38]
 [   0   28 2719    0   61]
 [   0  407   19 1303   11]
 [   0   26  587    0 3129]]
Báo cáo phân loại:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2842
           1       0.81      0.97      0.88      2034
           2       0.81      0.97      0.88      2808
           3       1.00      0.75      0.86      1740
           4       0.97      0.84      0.90      3742

    accuracy                           0.91     13166
   macro avg       0.92      0.90      0.90     13166
weighted avg       0.92      0.91      0.91     13166

